In [1]:
# ===========================================
# Install Required Packages
# ===========================================
! pip install -U torch transformers datasets scikit-learn langchain langchain-community

   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ------------------------------ --------- 0.8/1.0 MB 56.1 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ----------------------------- ---------- 1.8/2.5 MB 21.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 21.3 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 21.3 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 2.3 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 2.2 MB/s eta 0:00:01
   ------------------------------------- -- 2.4/2.5 MB 2.2 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 8.8 MB/s eta 0:00:00
   --------------------------


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import zipfile
import os
import joblib
import torch
from transformers import AutoTokenizer
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any

# --- 1. Ask user for paths ---
print("Enter full paths for model, scaler, and tokenizer directory.")
model_path = input("Path to model file (.pkl): ").strip()
numeric_scaler_path = input("Path to numeric_scaler.pkl: ").strip()
tokenizer_dir = input("Path to tokenizer directory: ").strip()

# --- 2. Validate paths ---
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at: {model_path}")
if not os.path.exists(numeric_scaler_path):
    raise FileNotFoundError(f"Scaler file not found at: {numeric_scaler_path}")
if not os.path.exists(tokenizer_dir):
    raise FileNotFoundError(f"Tokenizer directory not found at: {tokenizer_dir}")

# --- 3. Load tokenizer, scaler, and model ---
tokenizer = AutoTokenizer.from_pretrained(tokenizer_dir)
numeric_scaler = joblib.load(numeric_scaler_path)
model = torch.load(model_path, map_location=torch.device("cpu"), weights_only=False)
model.eval()

# --- 4. Create LangChain LLM wrapper ---
class TrainedModelLLM(LLM):
    """LangChain LLM wrapper for a trained BERT+numeric features model."""

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """
        Args:
            prompt: The input text. Must include numeric values in this format:
                    'feedback: <text> | features: val1,val2,val3,...'
        Returns:
            Prediction as a string (class label or probability).
        """
        # Parse numeric features from prompt
        if "| features:" in prompt:
            text_part, num_part = prompt.split("| features:")
            text_input = text_part.replace("feedback:", "").strip()
            numeric_strs = [x.strip() for x in num_part.strip().split(",")]
            numeric_values = list(map(float, numeric_strs))
        else:
            raise ValueError("Prompt must include '| features: num1,num2,...'")

        # Tokenize text
        encoded = tokenizer(
            text_input,
            truncation=True,
            padding=True,
            return_tensors="pt"
        )

        # Scale numeric features
        numeric_scaled = torch.tensor(
            numeric_scaler.transform([numeric_values]),
            dtype=torch.float
        )

        # Run inference
        with torch.no_grad():
            output = model(
                input_ids=encoded["input_ids"],
                attention_mask=encoded["attention_mask"],
                numeric_tensor=numeric_scaled
            )

        # Determine predicted class
        if isinstance(output, torch.Tensor):
            pred_class = output.argmax(dim=1).item()
        else:
            logits = getattr(
                output,
                "logits",
                output[0] if isinstance(output, (list, tuple)) else output
            )
            pred_class = logits.argmax(dim=1).item()

        return str(pred_class)

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": "Custom BERT + numeric model"}

    @property
    def _llm_type(self) -> str:
        return "custom_bert_numeric_llm"


# --- 5. Test the LLM wrapper ---
llm = TrainedModelLLM()
print("Enter Prompt")
prompt = ""
prediction = llm(prompt)
print("Prediction:", prediction)


Enter full paths for model, scaler, and tokenizer directory.
